<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/Sem%C3%A1foro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Follow-up SPS DC03 (3).xlsx to Follow-up SPS DC03 (3).xlsx
User uploaded file "Follow-up SPS DC03 (3).xlsx" with length 17741793 bytes


In [ ]:
r_a ='Follow-up SPS DC03 (3).xlsx'
df = pd.read_excel(r_a)
print(df.head(2))


           PO  Posición PO Tipo de Envio     SO KMEX  Posición KMEX  \
0  4512883075           10           SL3   1259948.0             10   
1  4512889998           90           SL3  53868808.0          10780   

  Fecha de PO Proveedor/Centro suministrador    Material  \
0  2025-01-02           650100     KRONES AG  0907308319   
1  2025-01-07           650100     KRONES AG  0902125801   

                                Texto breve Grupo de artículos  ...  \
0  Change of name plates, cable tags and do               VW00  ...   
1                  Verbindungskabel M8 20 m          EI0010200  ...   

   Fecha entrada Aduana  Fecha pago pedimento  Aereo  \
0                   NaN                   NaN    NaN   
1                   NaN                   NaN    NaN   

  Fecha pago pedimento aereo          Key2   Fecha EM  Cantidad EM  \
0                        NaN  451288307510        NaT          NaN   
1                        NaN  451288999890 2025-01-24          1.0   

           

In [ ]:
print(df.columns)

Index(['PO', 'Posición PO', 'Tipo de Envio', 'SO KMEX', 'Posición KMEX',
       'Fecha de PO', 'Proveedor/Centro suministrador', 'Material',
       'Texto breve', 'Grupo de artículos', 'Centro', 'Almacén',
       'Cantidad de pedido', 'Unidad medida pedido', 'Precio neto', 'Moneda',
       'SO KAG', 'Key', 'Delivery Note', 'EXW Estimado', 'EXW Real',
       'Diferencia dias', 'Cantidad entregada', 'Diferencia cantidad',
       'Consol ', 'Pedimento ', 'Fecha entrada Aduana', 'Fecha pago pedimento',
       'Aereo', 'Fecha pago pedimento aereo', 'Key2', 'Fecha EM',
       'Cantidad EM', 'Key3', 'Remision', 'Fecha remision'],
      dtype='object')


In [ ]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

st.title("Dashboard Semáforo de Fechas PO vs EXW Real")
st.write("Carga tu archivo Excel para visualizar el semáforo de fechas:")

archivo = st.file_uploader("Sube tu archivo Excel", type=["xlsx"])

if archivo:
    df = pd.read_excel(archivo)

    # Convertir a fecha
    df['Fecha de PO'] = pd.to_datetime(df['Fecha de PO'], errors='coerce')
    df['EXW Real'] = pd.to_datetime(df['EXW Real'], errors='coerce')

    # Calcular diferencia de días
    df['Diferencia días calculada'] = (df['EXW Real'] - df['Fecha de PO']).dt.days

    # Semáforo
    def status_semáforo(dias):
        if pd.isna(dias):
            return 'Sin fecha'
        elif 0 <= dias <= 6:
            return 'En tiempo'
        elif dias >= 7:
            return 'On Delay'
        else:
            return 'Revisar'

    df['Semáforo'] = df['Diferencia días calculada'].apply(status_semáforo)

    # Tabla con semáforo
    st.subheader("Tabla con Semáforo")
    st.dataframe(df[['Fecha de PO', 'EXW Real', 'Diferencia días calculada', 'Semáforo']])

    # Gague chart con Plotly
    st.subheader("Indicador Gauge: % En Tiempo")

    total_validos = df['Semáforo'].isin(['En tiempo', 'On Delay']).sum()
    en_tiempo = (df['Semáforo'] == 'En tiempo').sum()

    if total_validos > 0:
        porcentaje_en_tiempo = en_tiempo / total_validos * 100
    else:
        porcentaje_en_tiempo = 0

    fig = go.Figure(go.Indicator(
        mode = "gauge+number+delta",
        value = porcentaje_en_tiempo,
        title = {'text': "% En Tiempo (0-6 días)"},
        gauge = {
            'axis': {'range': [None, 100]},
            'steps': [
                {'range': [0, 60], 'color': "#F5B7B1"},    # rojo
                {'range': [60, 85], 'color': "#F9E79F"},   # amarillo
                {'range': [85, 100], 'color': "#ABEBC6"}   # verde
            ],
            'threshold': {
                'line': {'color': "black", 'width': 4},
                'thickness': 0.75,
                'value': porcentaje_en_tiempo
            }
        }
    ))

    st.plotly_chart(fig)

    st.info("Dashboard generado por Kai 🤖")

else:
    st.warning("Sube un archivo para ver el dashboard.")
